In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob

In [2]:
path_to_neuron_stats = r'E:\CIT_WAY\csvs'
neuron_stats_csv_name = 'neuron_stats'

path_to_ts = r'E:\CIT_WAY\csvs\spikes_time_series'

In [3]:
file_out_dir = r'E:\CIT_WAY\csvs'
file_out_name_ts = 'all_neurons_ts_with_clusters.csv'

In [4]:
def load_spikes_df(parent_dir):
    sub_dirs= os.listdir(parent_dir)
    df_list = []
    for sub_dir in sub_dirs:
        file_path = glob(os.path.join(parent_dir, sub_dir, '*.csv'))[0]  # should be only one csv file
        df_list.append(pd.read_csv(file_path))
    return pd.concat(df_list)

In [5]:
def load_neuron_stats(path, csv_name):
    if not csv_name.endswith('.csv'):
        csv_name = ''.join([csv_name, '.csv'])
    return pd.read_csv(os.path.join(path, csv_name))

In [6]:
def neuron_category_mapper(row):
    '''
    This is the function we use to categorise neurons as either slow or fast, and as either regular or irregular
    It categorises neurons firing slower than 4.5 Hz as slow and neurons with a CV ISI of less than 0.55 as regular
    '''
    if row['Firing Rate'] <= 4.5:
        rate = 'slow'
    else:
        rate = 'fast'
    if row['CV ISI'] <= 0.61:
        reg = 'regular'
    else:
        reg = 'irregular'
    return ' '.join([rate, reg])

In [7]:
df_ts = load_spikes_df(path_to_ts)
df_stats = load_neuron_stats(path=path_to_neuron_stats, csv_name=neuron_stats_csv_name)

In [8]:
df_ts.head()

,spike_cluster,time,condition,recording
0,99,0.015833,Baseline,2018-05-01_01
1,57,0.023667,Baseline,2018-05-01_01
2,119,0.023767,Baseline,2018-05-01_01
3,97,0.026600,Baseline,2018-05-01_01
4,97,0.048833,Baseline,2018-05-01_01


In [9]:
df_stats.head()

,spike_cluster,CV ISI,Firing Rate,recording,SW_peak,SW_return,SW_troff,base_min_amp,min_max_amp,spike_type
0,0,0.870878,0.806234,2018-05-01_01,0.133333,0.900000,0.433333,161.232773,228.980727,up_down_up
1,4,0.832558,0.668532,2018-05-01_01,NaN,0.800000,0.433333,224.799429,NaN,down_up
2,6,0.931899,0.622853,2018-05-01_01,NaN,0.566667,0.433333,158.659843,NaN,down_up
3,9,0.609180,3.513685,2018-05-01_01,0.066667,0.466667,0.300000,291.783264,326.794909,up_down_up
4,13,0.530543,1.160064,2018-05-01_01,NaN,0.833333,0.366667,479.876392,NaN,down_up


In [10]:
df_stats['category'] = df_stats.apply(neuron_category_mapper, axis=1)
df_stats.head()

,spike_cluster,CV ISI,Firing Rate,recording,SW_peak,SW_return,SW_troff,base_min_amp,min_max_amp,spike_type,category
0,0,0.870878,0.806234,2018-05-01_01,0.133333,0.900000,0.433333,161.232773,228.980727,up_down_up,slow irregular
1,4,0.832558,0.668532,2018-05-01_01,NaN,0.800000,0.433333,224.799429,NaN,down_up,slow irregular
2,6,0.931899,0.622853,2018-05-01_01,NaN,0.566667,0.433333,158.659843,NaN,down_up,slow irregular
3,9,0.609180,3.513685,2018-05-01_01,0.066667,0.466667,0.300000,291.783264,326.794909,up_down_up,slow regular
4,13,0.530543,1.160064,2018-05-01_01,NaN,0.833333,0.366667,479.876392,NaN,down_up,slow regular


In [11]:
df_ts_merge = pd.merge(left=df_ts,
                         right=df_stats[['recording', 'category', 'spike_cluster']],
                        left_on=['recording', 'spike_cluster'],
                        right_on=['recording', 'spike_cluster'],
                        how='right')

In [13]:
df_ts_merge.to_csv(os.path.join(file_out_dir, file_out_name_ts), index=False)

In [20]:
df_stats.to_csv(os.path.join(file_out_dir, neuron_stats_csv_name) + '.csv', 
                index=False)